In [11]:
import pandas as pd
import requests
import zipfile
import tarfile

## Reading corpus data

In [ ]:
# how to read data file using pandas from github repo
url= 'https://github.com/scrosseye/CLEAR-Corpus/blob/main/CLEAR_corpus_final.xlsx?raw=True'
myfile = requests.get(url)

df=pd.read_excel(url, sheet_name='Data', engine='openpyxl')
print(df.info())


## Corpus of Sentences rated with Human Complexity Judgments

In [ ]:
file_path = './../data/complexity_ds_en.csv'
df = pd.read_csv(file_path)
print(df.info())

## REALEC Corpus

In [21]:
# open tar.gz file name Exam2016.tar.gz in data folder and print all files in that that have .txt and .json extension, then store them in a list
# all content of txt file as string and from json file read the content of key 'ielts', 'text' and store them as pandas dataframe

file_path = './../data/Exam2016.tar.gz'
count = 0
dict_text = {}
dict_json = {}
data = []
with tarfile.open(file_path, "r:gz") as tar:
    count += 1
    for tarinfo in tar:
        if tarinfo.name.endswith(".txt") or tarinfo.name.endswith(".json"):
            file_name = tarinfo.name
            file_name = file_name.split('/')[1].split('.')[0]
            with tar.extractfile(tarinfo) as f:
                if tarinfo.name.endswith(".txt"):
                    str_sentence = f.read()
                    dict_text[file_name] = str_sentence
                else:
                    json_data = pd.read_json(f, orient='index').T
                    dict_json[file_name] = json_data


for key in dict_text.keys():
    datum = []
    sentence = dict_text[key]
    json_data = dict_json[key]
    datum = [sentence, json_data['ielts'][0], json_data['CEFR_level'][0], json_data['work_type'][0], json_data['year'][0]]
    data.append(datum)

df_pd = pd.DataFrame(data, columns=['sentence', 'ielts', 'CEFR_level', 'work_type', 'year'])
df_pd = df_pd[df_pd['CEFR_level'] != '']
df_pd.to_csv('./../data/Exam2016.csv', index=False, header=True)
            